# Intro to firestorm

Following the standard [tutorial](https://cloud.google.com/firestore/docs/quickstart-servers)


In [ ]:
from google.cloud import firestore

import itertools

### Authentication 

Authentication can be a bitch. Either:
- `export` account credentials as environment veriable, or;
- make the credentials explicit to your project. 

The latter is preferred, see [docs](https://googleapis.github.io/google-cloud-python/latest/core/auth.html).

Also make sure the service account has the right [Firebase access rights](https://firebase.google.com/docs/projects/iam/permissions)

In [ ]:
db = firestore.Client.from_service_account_json("../credentials/stairway-firestore-key.json")

### Writing data

As there is no schema you can write anything, there are no rules to enforce a specific format. The below code for example creates a new document in a new database if there isn't any yet. If it is existing, it updates the properties instead.

The documents are basically JSON, so use key-value pairs.

To create or overwrite a single document, use the `set()` method:

In [ ]:
# adding data
doc_ref = db.collection("test").document('alovelace')
doc_ref.set({
    u'first': u'Ada',
    u'last': u'Lovelace2',
    u'born': 1815
})

### Reading data

In [ ]:
# read data
users_ref = db.collection("destinations-old")
docs = users_ref.get()

for doc in itertools.islice(docs, 5):
    print(u'{} => {}'.format(doc.id, doc.to_dict()['name']))

Filter using `where` clause, see [examples](https://firebase.google.com/docs/firestore/query-data/queries). 

In [ ]:
query = (
    db
    .collection("destinations-old")
    .where('AF', '==', 1)
    .get()
)

for doc in query:
    print(u'{} => {}'.format(doc.id, doc.to_dict()['name']))


As there's no random select function, a [solution](https://stackoverflow.com/questions/46798981/firestore-how-to-get-random-documents-in-a-collection) could be to add a random index to Firestore and limit the result to 1. For demonstration purposes, let's use the feature `osp_importance` as a proxy for random.

In [ ]:
query = (
    db
    .collection("destinations-old")
    .where('AF', '==', 1)
#     .order_by('osp_importance')
    .limit(3)
    .get()
)

for doc in itertools.islice(query, 2):
    print(u'{} => {}'.format(doc.id, doc.to_dict()['name']))

For returning a single document through the API we can use `to_dict()` to return the relevant JSON.

You can also return a list of documents. This is relevant for the `explore` tab wherein the user gets to see multiple destinations:

In [ ]:
query = (
    db
    .collection("destinations-old")
    .limit(5)
    .get()
)

{
    "Destinations": [item.to_dict() for item in list(query)]
}

### Deleting data

See documentation in tutorial

In [ ]:
# removing a single document
db.collection("test").document("alovelace").delete()

Deleting a collection requires iterating over the documents in it.

In [ ]:
def delete_collection(coll_ref, batch_size):
    docs = coll_ref.limit(10).get()
    deleted = 0

    for doc in docs:
        print(u'Deleting doc {} => {}'.format(doc.id, doc.to_dict()))
        doc.reference.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)

In [ ]:
delete_collection(db.collection("test"), 5)

Done.